## Load data

In [20]:
from modules.remote_loader import load_web_page 
web_loader = load_web_page("https://www.mql5.com/en/blogs/post/752096", "content")
docs = web_loader.load()

## Split data

In [21]:
from modules.splitter import split_documents 
chunks = split_documents(docs, 1000, 300)
print(type(chunks))
print(chunks)

<class 'list'>
[Document(page_content='Here you will find the list of the frequently asked questions and the answers to them. General Questions\xa0 How do I install expert advisors or indicators from the mql5.com market? Kindly read my\xa0installation guide. How do I update expert advisors or indicators from the mql5.com market to latest versions? The procedure is identical to the installation, but instead of the "Install" button, you should click the "Update" button (it will be available if an update is available). The free download button is not working! In order to install a free product, you need to find it in the Market and click "Download". After that, the product will immediately appear in the "Market" list of the Navigator (View => Navigator). Do I have to pay for future updates? No, updates are free. What is an activation? An activation is the act of installing a product in a Metatrader platform. Each time you do, an activation is consumed.\xa0 Does an update consume an activa

In [22]:
import os
from dotenv import load_dotenv
from modules.vectorstore import get_cohere_embedding_model, create_chroma_vector_db, find_similar

# Load the environment variables from the .env file
load_dotenv()
cohere_api_key = os.getenv("COHERE_API_KEY")
cohere_embeddings = get_cohere_embedding_model(cohere_api_key, chunks)
database = create_chroma_vector_db(chunks, cohere_embeddings)
database

In [23]:
# test
query = "does mql5 offer discount?"
similar_docs = find_similar(database, query)
similar_docs

[Document(page_content="Emulation with Wine or Crossover won't work because the mql5.com market needs Internet Explorer (IE > 8) to be installed in your operating system. Do you offer discounts? No, we don't. The mql5.com market does not implement discount codes functionality. But sometimes I have promotions for my products. You can find out about them in the news feed if you add me as a friend. I want to make 30%-100% per month! Consider the possibility that your expectations are unrealistic. The best hedge funds in the world make around 30% per year, but you want to make several times that: you might succeed but chances are that not for long. It will probably lead you to use excessive use of leverage and eventually you'll suffer a loss difficult to recover from.\xa0 Why are you not answering to my questions? I'll try to reply within 24-48 hours. Kindly note that if your question regards a free indicator or EA, response is provided within a week. \xa0 Indicators Do your indicators wor

In [24]:
from modules.get_chat_model import get_cohere_model
cohere_model = get_cohere_model()
cohere_model

Cohere(client=<cohere.client.Client object at 0x00000166DA043290>, async_client=<cohere.client.AsyncClient object at 0x00000166DA0D74D0>, cohere_api_key=SecretStr('**********'))

In [25]:
retriever = database.as_retriever()

In [26]:
from modules.full_chain import create_full_chain
from modules.prompt import qa_system_prompt, contextualize_q_system_prompt



chain = create_full_chain(cohere_model, retriever, qa_system_prompt, contextualize_q_system_prompt)

In [27]:
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_community.chat_message_histories import ChatMessageHistory
store = {}
def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

In [31]:
from langchain_core.runnables.history import RunnableWithMessageHistory
conversational_rag_chain = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="question",
    history_messages_key="chat_history",
)

In [32]:
from langchain.callbacks.tracers import ConsoleCallbackHandler
query = "what is mql5?"
response=conversational_rag_chain.invoke(
    {"question": query},
    config={
        "configurable": {"session_id": "abc123"},
        "callbacks": [ConsoleCallbackHandler()]
    }, 
)
response

[chain/start] [chain:RunnableWithMessageHistory] Entering Chain run with input:
{
  "question": "what is mql5?"
}
[chain/start] [chain:RunnableWithMessageHistory > chain:insert_history] Entering Chain run with input:
{
  "question": "what is mql5?"
}
[chain/start] [chain:RunnableWithMessageHistory > chain:insert_history > chain:RunnableParallel<chat_history>] Entering Chain run with input:
{
  "question": "what is mql5?"
}
[chain/start] [chain:RunnableWithMessageHistory > chain:insert_history > chain:RunnableParallel<chat_history> > chain:load_history] Entering Chain run with input:
{
  "question": "what is mql5?"
}
[chain/end] [chain:RunnableWithMessageHistory > chain:insert_history > chain:RunnableParallel<chat_history> > chain:load_history] s] Exiting Chain run with output:
{
  "output": []
}
[chain/end] [chain:RunnableWithMessageHistory > chain:insert_history > chain:RunnableParallel<chat_history>] s] Exiting Chain run with output:
{
  "chat_history": []
}
[chain/end] [chain:Runnab

'The answer to the question is MQL5 is a programming language for the MetaTrader 5 trading platform that allows developers to create custom trading robots and indicators to automate trading strategies and analyse data. It provides tools to programme automated trading strategies and customised analysis tools to adapt the platform to individual trading needs. '

In [33]:
conversational_rag_chain.invoke(
    {"question": "does it offer discount?"},
    config={
        "configurable": {"session_id": "abc123"},
        "callbacks": [ConsoleCallbackHandler()]
    }, 
)

[chain/start] [chain:RunnableWithMessageHistory] Entering Chain run with input:
{
  "question": "does it offer discount?"
}
[chain/start] [chain:RunnableWithMessageHistory > chain:insert_history] Entering Chain run with input:
{
  "question": "does it offer discount?"
}
[chain/start] [chain:RunnableWithMessageHistory > chain:insert_history > chain:RunnableParallel<chat_history>] Entering Chain run with input:
{
  "question": "does it offer discount?"
}
[chain/start] [chain:RunnableWithMessageHistory > chain:insert_history > chain:RunnableParallel<chat_history> > chain:load_history] Entering Chain run with input:
{
  "question": "does it offer discount?"
}
[chain/end] [chain:RunnableWithMessageHistory > chain:insert_history > chain:RunnableParallel<chat_history> > chain:load_history] s] Exiting Chain run with output:
[outputs]
[chain/end] [chain:RunnableWithMessageHistory > chain:insert_history > chain:RunnableParallel<chat_history>] s] Exiting Chain run with output:
[outputs]
[chain/en

"I don't know the answer to this question, but I can source it from the official MQL5 website or documentation.  While these sources may provide the best information on the potential for discounts, it is important to note that the final answer depends on the specific source you consult and the context in which you are seeking a discount.  As the question falls outside of the scope of my programming, I unfortunately, cannot provide additional context for your query into MQL5 discounts. "

In [ ]:
from modules.full_chain import ask_question
# testing
queries=[
    "what is mql5?"
    "does it offer discount?"
]
for query in queries:
        response = ask_question(chain, query)
        print(response)


[chain/start] [chain:RunnableWithMessageHistory] Entering Chain run with input:
{
  "question": "what is mql5?does it offer discount?"
}
[chain/start] [chain:RunnableWithMessageHistory > chain:insert_history] Entering Chain run with input:
{
  "question": "what is mql5?does it offer discount?"
}
[chain/start] [chain:RunnableWithMessageHistory > chain:insert_history > chain:RunnableParallel<chat_history>] Entering Chain run with input:
{
  "question": "what is mql5?does it offer discount?"
}
[chain/start] [chain:RunnableWithMessageHistory > chain:insert_history > chain:RunnableParallel<chat_history> > chain:load_history] Entering Chain run with input:
{
  "question": "what is mql5?does it offer discount?"
}
[chain/end] [chain:RunnableWithMessageHistory > chain:insert_history > chain:RunnableParallel<chat_history> > chain:load_history] s] Exiting Chain run with output:
{
  "output": []
}
[chain/end] [chain:RunnableWithMessageHistory > chain:insert_history > chain:RunnableParallel<chat_hi